In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e6/sample_submission.csv
/kaggle/input/playground-series-s4e6/train.csv
/kaggle/input/playground-series-s4e6/test.csv


# **IMPORTS AND DATA LOADING**

**Imports**

In [2]:
# FOR PREPROCESSING
from sklearn.preprocessing import LabelEncoder, StandardScaler

# TRAINING TESTING
from sklearn.model_selection import train_test_split, cross_val_score

In [3]:
# MODELS 
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import xgboost as xgb

In [4]:
#FOR HYPERPARAMETER TUNING
import optuna

In [5]:
from sklearn.metrics import accuracy_score, classification_report

**Data**

In [6]:
train = pd.read_csv("/kaggle/input/playground-series-s4e6/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s4e6/test.csv")

# **BEGINNER LEVEL**
> **The approach is just to create a trainable data and using a simple model, for this I used Decision Tree because we had both categorical(dtype = 'int64') as well as the continuos data(dtype = 'float64') and for such combination of data decision tree works very well, we don't even need to normalize data, or do any kind of encodings for Target variable here as well.** <br>
*Note : Here I didn't used categorical data as category columns it is used as it is, that is as numerical data.*
## IMP INFO
> ****For more advanced solution please go down to Comparison's of models done below

# **DATA ANALYSIS**

In [7]:
train.describe()

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
count,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,...,76518.00000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000,76518.000000
mean,38258.500000,1.111934,16.054419,1.644410,9001.286377,0.915314,3.658760,132.378766,1.226600,19.837633,...,0.05796,0.137053,5.933414,7.234468,4.007201,9.626085,0.062443,11.520340,1.228218,-0.080921
std,22088.988286,0.441669,16.682337,1.229645,1803.438531,0.278416,8.623774,10.995328,3.392183,15.399456,...,0.40849,0.933830,1.627182,3.503040,2.772956,5.546035,0.462107,2.653375,1.398816,2.251382
min,0.000000,1.000000,1.000000,0.000000,33.000000,0.000000,1.000000,95.000000,1.000000,1.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.600000,-0.800000,-4.060000
25%,19129.250000,1.000000,1.000000,1.000000,9119.000000,1.000000,1.000000,125.000000,1.000000,1.000000,...,0.00000,0.000000,5.000000,6.000000,1.000000,10.000000,0.000000,9.400000,0.300000,-1.700000
50%,38258.500000,1.000000,17.000000,1.000000,9254.000000,1.000000,1.000000,133.100000,1.000000,19.000000,...,0.00000,0.000000,6.000000,7.000000,5.000000,12.142857,0.000000,11.100000,1.400000,0.320000
75%,57387.750000,1.000000,39.000000,2.000000,9670.000000,1.000000,1.000000,140.000000,1.000000,37.000000,...,0.00000,0.000000,6.000000,9.000000,6.000000,13.244048,0.000000,12.700000,2.600000,1.790000
max,76517.000000,6.000000,53.000000,9.000000,9991.000000,1.000000,43.000000,190.000000,109.000000,44.000000,...,12.00000,19.000000,23.000000,33.000000,20.000000,18.000000,12.000000,16.200000,3.700000,3.510000


In [8]:
train.isnull().sum()

id                                                0
Marital status                                    0
Application mode                                  0
Application order                                 0
Course                                            0
Daytime/evening attendance                        0
Previous qualification                            0
Previous qualification (grade)                    0
Nacionality                                       0
Mother's qualification                            0
Father's qualification                            0
Mother's occupation                               0
Father's occupation                               0
Admission grade                                   0
Displaced                                         0
Educational special needs                         0
Debtor                                            0
Tuition fees up to date                           0
Gender                                            0
Scholarship 

In [9]:
test

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,76518,1,1,1,9500,1,1,141.0,1,3,...,0,0,8,0,0,0.000000,0,13.9,-0.3,0.79
1,76519,1,1,1,9238,1,1,128.0,1,1,...,0,0,6,6,6,13.500000,0,11.1,0.6,2.02
2,76520,1,1,1,9238,1,1,118.0,1,1,...,0,0,6,11,5,11.000000,0,15.5,2.8,-4.06
3,76521,1,44,1,9147,1,39,130.0,1,1,...,0,3,8,14,5,11.000000,0,8.9,1.4,3.51
4,76522,1,39,1,9670,1,1,110.0,1,1,...,0,0,6,9,4,10.666667,2,7.6,2.6,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51007,127525,1,1,2,171,1,1,128.0,1,38,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51008,127526,2,39,1,9119,1,19,133.1,1,19,...,0,0,5,5,0,0.000000,0,9.4,-0.8,-3.12
51009,127527,1,1,1,171,1,1,127.0,1,1,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51010,127528,1,1,3,9773,1,1,132.0,1,19,...,0,0,6,9,3,13.000000,0,7.6,2.6,0.32


In [10]:
train.set_index("id",inplace=True)

In [11]:
train.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
id,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,9238,1,1,126.0,1,1,19,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,17,1,9238,1,1,125.0,1,19,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,1,17,2,9254,1,1,137.0,1,3,19,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,1,1,3,9500,1,1,131.0,1,19,3,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,1,1,2,9500,1,1,132.0,1,19,37,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate


In [12]:
for i in train.columns:
    if train[i].dtype == 'int64':
        print(f"{i} --> {train[i].dtype}")

Marital status --> int64
Application mode --> int64
Application order --> int64
Course --> int64
Daytime/evening attendance --> int64
Previous qualification --> int64
Nacionality --> int64
Mother's qualification --> int64
Father's qualification --> int64
Mother's occupation --> int64
Father's occupation --> int64
Displaced --> int64
Educational special needs --> int64
Debtor --> int64
Tuition fees up to date --> int64
Gender --> int64
Scholarship holder --> int64
Age at enrollment --> int64
International --> int64
Curricular units 1st sem (credited) --> int64
Curricular units 1st sem (enrolled) --> int64
Curricular units 1st sem (evaluations) --> int64
Curricular units 1st sem (approved) --> int64
Curricular units 1st sem (without evaluations) --> int64
Curricular units 2nd sem (credited) --> int64
Curricular units 2nd sem (enrolled) --> int64
Curricular units 2nd sem (evaluations) --> int64
Curricular units 2nd sem (approved) --> int64
Curricular units 2nd sem (without evaluations) --

In [13]:
for i in train.columns:
    if train[i].dtype == 'float64':
        print(f"{i} --> {train[i].dtype}")

Previous qualification (grade) --> float64
Admission grade --> float64
Curricular units 1st sem (grade) --> float64
Curricular units 2nd sem (grade) --> float64
Unemployment rate --> float64
Inflation rate --> float64
GDP --> float64


# **FEATURE TYPE**

## CATEGORICAL DATA
* Marital status --> int64
* Application mode --> int64
* Course --> int64
* Daytime/evening attendance --> int64
* Previous qualification --> int64
* Nacionality --> int64
* Mother's qualification --> int64
* Father's qualification --> int64
* Mother's occupation --> int64
* Father's occupation --> int64
* Displaced --> int64
* Educational special needs --> int64
* Debtor --> int64
* Tuition fees up to date --> int64
* Gender --> int64
* Scholarship holder --> int64
* International --> int64


## CONTINUOS DATA
* Application order --> int64
* Previous qualification (grade) --> float64
* Admission grade --> float64
* Age at enrollment --> int64
* Curricular units 1st sem (credited) --> int64
* Curricular units 1st sem (enrolled) --> int64
* Curricular units 1st sem (evaluations) --> int64
* Curricular units 1st sem (approved) --> int64
* Curricular units 1st sem (without evaluations) --> int64
* Curricular units 1st sem (grade) --> float64
* Curricular units 2nd sem (credited) --> int64
* Curricular units 2nd sem (enrolled) --> int64
* Curricular units 2nd sem (evaluations) --> int64
* Curricular units 2nd sem (approved) --> int64
* Curricular units 2nd sem (without evaluations) --> int64
* Curricular units 2nd sem (grade) --> float64
* Unemployment rate --> float64
* Inflation rate --> float64
* GDP --> float64


In [14]:
cat_data = [
    "Marital status",
    "Application mode",
    "Course",
    "Daytime/evening attendance",
    "Previous qualification",
    "Nacionality",
    "Mother's qualification",
    "Father's qualification",
    "Mother's occupation",
    "Father's occupation",
    "Displaced",
    "Educational special needs",
    "Debtor",
    "Tuition fees up to date",
    "Gender",
    "Scholarship holder",
    "International"
]

In [15]:
cont_data = [
    "Application order",
    "Previous qualification (grade) ",
    "Admission grade" ,
    "Age at enrollment",
    "Curricular units 1st sem (credited)",
    "Curricular units 1st sem (enrolled)",
    "Curricular units 1st sem (evaluations)",
   " Curricular units 1st sem (approved)",
    "Curricular units 1st sem (without evaluations)",
    "Curricular units 1st sem (grade) ",
    "Curricular units 2nd sem (credited)",
    "Curricular units 2nd sem (enrolled)",
    "Curricular units 2nd sem (evaluations)",
    "Curricular units 2nd sem (approved)",
    "Curricular units 2nd sem (without evaluations)",
    "Curricular units 2nd sem (grade) ",
    "Unemployment rate ",
    "Inflation rate ",
    "GDP "
]

# DATA PREPROCESSING

In [16]:
for i in cat_data:
    train[i] = train[i].astype('category')

In [17]:
train

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
id,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,9238,1,1,126.0,1,1,19,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,17,1,9238,1,1,125.0,1,19,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,1,17,2,9254,1,1,137.0,1,3,19,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,1,1,3,9500,1,1,131.0,1,19,3,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,1,1,2,9500,1,1,132.0,1,19,37,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,1,17,1,9254,1,1,121.0,1,19,1,...,0,6,8,5,10.600000,0,13.9,-0.3,0.79,Graduate
76514,1,1,6,9254,1,1,125.0,1,1,38,...,0,6,9,6,13.875000,0,9.4,-0.8,-3.12,Graduate
76515,5,17,1,9085,1,1,138.0,1,37,37,...,0,5,8,5,11.400000,1,9.4,-0.8,-3.12,Enrolled


In [18]:
ohe = pd.get_dummies(train[cat_data],dtype = 'int64')

In [19]:
ohe

,Marital status_1,Marital status_2,Marital status_3,Marital status_4,Marital status_5,Marital status_6,Application mode_1,Application mode_2,Application mode_3,Application mode_4,...,Debtor_0,Debtor_1,Tuition fees up to date_0,Tuition fees up to date_1,Gender_0,Gender_1,Scholarship holder_0,Scholarship holder_1,International_0,International_1
id,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,0,0,0,1,0,0,0,...,1,0,0,1,1,0,0,1,1,0
1,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,1,0,1,0
2,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,1,1,0,1,0
3,1,0,0,0,0,0,1,0,0,0,...,1,0,0,1,1,0,0,1,1,0
4,1,0,0,0,0,0,1,0,0,0,...,1,0,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,1,1,0
76514,1,0,0,0,0,0,1,0,0,0,...,1,0,0,1,1,0,1,0,1,0
76515,0,0,0,0,1,0,0,0,0,0,...,1,0,0,1,1,0,1,0,1,0


In [20]:
df = pd.concat([train,ohe],axis=1)

In [21]:
df.columns

Index(['Marital status', 'Application mode', 'Application order', 'Course',
       'Daytime/evening attendance', 'Previous qualification',
       'Previous qualification (grade)', 'Nacionality',
       'Mother's qualification', 'Father's qualification',
       ...
       'Debtor_0', 'Debtor_1', 'Tuition fees up to date_0',
       'Tuition fees up to date_1', 'Gender_0', 'Gender_1',
       'Scholarship holder_0', 'Scholarship holder_1', 'International_0',
       'International_1'],
      dtype='object', length=309)

In [22]:
df

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Debtor_0,Debtor_1,Tuition fees up to date_0,Tuition fees up to date_1,Gender_0,Gender_1,Scholarship holder_0,Scholarship holder_1,International_0,International_1
id,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,9238,1,1,126.0,1,1,19,...,1,0,0,1,1,0,0,1,1,0
1,1,17,1,9238,1,1,125.0,1,19,19,...,1,0,0,1,1,0,1,0,1,0
2,1,17,2,9254,1,1,137.0,1,3,19,...,1,0,0,1,0,1,1,0,1,0
3,1,1,3,9500,1,1,131.0,1,19,3,...,1,0,0,1,1,0,0,1,1,0
4,1,1,2,9500,1,1,132.0,1,19,37,...,1,0,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,1,17,1,9254,1,1,121.0,1,19,1,...,1,0,0,1,1,0,0,1,1,0
76514,1,1,6,9254,1,1,125.0,1,1,38,...,1,0,0,1,1,0,1,0,1,0
76515,5,17,1,9085,1,1,138.0,1,37,37,...,1,0,0,1,1,0,1,0,1,0


In [23]:
a=["Marital status","Application mode","Course","Daytime/evening attendance","Previous qualification","Nacionality","Mother's qualification","Father's qualification","Mother's occupation","Father's occupation","Displaced","Educational special needs","Debtor","Tuition fees up to date","Gender","Scholarship holder","International"]

In [24]:
d = df.drop(a,axis=1)

In [25]:
d

,Application order,Previous qualification (grade),Admission grade,Age at enrollment,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),...,Debtor_0,Debtor_1,Tuition fees up to date_0,Tuition fees up to date_1,Gender_0,Gender_1,Scholarship holder_0,Scholarship holder_1,International_0,International_1
id,,,,,,,,,,,,,,,,,,,,,
0,1,126.0,122.6,18,0,6,6,6,14.500000,0,...,1,0,0,1,1,0,0,1,1,0
1,1,125.0,119.8,18,0,6,8,4,11.600000,0,...,1,0,0,1,1,0,1,0,1,0
2,2,137.0,144.7,18,0,6,0,0,0.000000,0,...,1,0,0,1,0,1,1,0,1,0
3,3,131.0,126.1,18,0,7,9,7,12.591250,0,...,1,0,0,1,1,0,0,1,1,0
4,2,132.0,120.1,18,0,7,12,6,12.933333,0,...,1,0,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,1,121.0,116.5,18,0,6,9,6,10.666667,0,...,1,0,0,1,1,0,0,1,1,0
76514,6,125.0,131.6,19,0,6,22,4,13.000000,0,...,1,0,0,1,1,0,1,0,1,0
76515,1,138.0,123.3,19,0,5,13,4,12.500000,2,...,1,0,0,1,1,0,1,0,1,0


In [26]:
features = d.drop("Target",axis=1)
target = d.Target

# TRAINING & TESTING USING DECISION TREE CLASSIFIER

In [27]:
X,x,Y,y = train_test_split(features,target,test_size = 0.2)

In [28]:
dct = DecisionTreeClassifier()

In [29]:
dct.fit(X,Y)

DecisionTreeClassifier()

In [30]:
y_pred = dct.predict(x)

In [31]:
print(accuracy_score(y,y_pred))

0.746144798745426


# PREDICTION ON TEST DATA

In [32]:
test.isnull().sum()

id                                                0
Marital status                                    0
Application mode                                  0
Application order                                 0
Course                                            0
Daytime/evening attendance                        0
Previous qualification                            0
Previous qualification (grade)                    0
Nacionality                                       0
Mother's qualification                            0
Father's qualification                            0
Mother's occupation                               0
Father's occupation                               0
Admission grade                                   0
Displaced                                         0
Educational special needs                         0
Debtor                                            0
Tuition fees up to date                           0
Gender                                            0
Scholarship 

In [33]:
to_test = test.drop("id",axis=1)

In [34]:
to_test.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,1,1,1,9500,1,1,141.0,1,3,1,...,0,0,8,0,0,0.000000,0,13.9,-0.3,0.79
1,1,1,1,9238,1,1,128.0,1,1,19,...,0,0,6,6,6,13.500000,0,11.1,0.6,2.02
2,1,1,1,9238,1,1,118.0,1,1,19,...,0,0,6,11,5,11.000000,0,15.5,2.8,-4.06
3,1,44,1,9147,1,39,130.0,1,1,19,...,0,3,8,14,5,11.000000,0,8.9,1.4,3.51
4,1,39,1,9670,1,1,110.0,1,1,37,...,0,0,6,9,4,10.666667,2,7.6,2.6,0.32


In [35]:
for i in cat_data:
    to_test[i] = to_test[i].astype('category')

In [36]:
ohe_test = pd.get_dummies(to_test[cat_data],dtype = 'int64')

In [37]:
ohe_test

,Marital status_1,Marital status_2,Marital status_3,Marital status_4,Marital status_5,Marital status_6,Application mode_1,Application mode_2,Application mode_3,Application mode_5,...,Debtor_0,Debtor_1,Tuition fees up to date_0,Tuition fees up to date_1,Gender_0,Gender_1,Scholarship holder_0,Scholarship holder_1,International_0,International_1
0,1,0,0,0,0,0,1,0,0,0,...,1,0,1,0,1,0,1,0,1,0
1,1,0,0,0,0,0,1,0,0,0,...,1,0,0,1,1,0,1,0,1,0
2,1,0,0,0,0,0,1,0,0,0,...,1,0,0,1,1,0,0,1,1,0
3,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,1,1,0
4,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51007,1,0,0,0,0,0,1,0,0,0,...,1,0,0,1,1,0,1,0,1,0
51008,0,1,0,0,0,0,0,0,0,0,...,0,1,1,0,0,1,1,0,1,0
51009,1,0,0,0,0,0,1,0,0,0,...,0,1,1,0,1,0,1,0,1,0
51010,1,0,0,0,0,0,1,0,0,0,...,0,1,1,0,1,0,1,0,1,0


> **We have less number of features here because few classes aren't used in columns.**

Let's view the lost columns

In [38]:
for i in ohe.columns:
    if i not in ohe_test.columns:
        print(i)
    

Application mode_4
Application mode_9
Application mode_12
Application mode_26
Course_39
Course_979
Previous qualification_36
Previous qualification_37
Nacionality_17
Mother's qualification_7
Mother's qualification_8
Mother's qualification_15
Mother's qualification_27
Mother's qualification_28
Father's qualification_15
Father's qualification_20
Father's qualification_23
Father's qualification_24
Father's qualification_44
Mother's occupation_11
Mother's occupation_38
Mother's occupation_101
Mother's occupation_103
Mother's occupation_127
Mother's occupation_163
Mother's occupation_172
Father's occupation_12
Father's occupation_13
Father's occupation_19
Father's occupation_22
Father's occupation_39
Father's occupation_96
Father's occupation_132
Father's occupation_148
Father's occupation_191


> **Getting back lost columns with value 0**

In [39]:
ohe_test = ohe_test.reindex(columns=ohe.columns, fill_value=0)

In [40]:
ohe_test

,Marital status_1,Marital status_2,Marital status_3,Marital status_4,Marital status_5,Marital status_6,Application mode_1,Application mode_2,Application mode_3,Application mode_4,...,Debtor_0,Debtor_1,Tuition fees up to date_0,Tuition fees up to date_1,Gender_0,Gender_1,Scholarship holder_0,Scholarship holder_1,International_0,International_1
0,1,0,0,0,0,0,1,0,0,0,...,1,0,1,0,1,0,1,0,1,0
1,1,0,0,0,0,0,1,0,0,0,...,1,0,0,1,1,0,1,0,1,0
2,1,0,0,0,0,0,1,0,0,0,...,1,0,0,1,1,0,0,1,1,0
3,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,1,1,0
4,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51007,1,0,0,0,0,0,1,0,0,0,...,1,0,0,1,1,0,1,0,1,0
51008,0,1,0,0,0,0,0,0,0,0,...,0,1,1,0,0,1,1,0,1,0
51009,1,0,0,0,0,0,1,0,0,0,...,0,1,1,0,1,0,1,0,1,0
51010,1,0,0,0,0,0,1,0,0,0,...,0,1,1,0,1,0,1,0,1,0


In [41]:
df_test = pd.concat([to_test,ohe_test],axis=1)

In [42]:
a=["Marital status","Application mode","Course","Daytime/evening attendance","Previous qualification","Nacionality","Mother's qualification","Father's qualification","Mother's occupation","Father's occupation","Displaced","Educational special needs","Debtor","Tuition fees up to date","Gender","Scholarship holder","International"]

In [43]:
d_test = df_test.drop(a,axis=1)

In [44]:
d_test

,Application order,Previous qualification (grade),Admission grade,Age at enrollment,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),...,Debtor_0,Debtor_1,Tuition fees up to date_0,Tuition fees up to date_1,Gender_0,Gender_1,Scholarship holder_0,Scholarship holder_1,International_0,International_1
0,1,141.0,152.1,18,0,7,0,0,0.000000,0,...,1,0,1,0,1,0,1,0,1,0
1,1,128.0,116.5,19,0,6,7,6,14.857143,0,...,1,0,0,1,1,0,1,0,1,0
2,1,118.0,114.2,18,0,6,11,6,12.000000,0,...,1,0,0,1,1,0,0,1,1,0
3,1,130.0,130.0,23,2,6,15,5,11.500000,0,...,1,0,0,1,1,0,0,1,1,0
4,1,110.0,106.0,26,0,6,9,3,11.000000,0,...,1,0,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51007,2,128.0,124.7,19,0,0,0,0,0.000000,0,...,1,0,0,1,1,0,1,0,1,0
51008,1,133.1,140.0,33,0,5,6,0,0.000000,0,...,0,1,1,0,0,1,1,0,1,0
51009,1,127.0,120.4,20,0,0,0,0,0.000000,0,...,0,1,1,0,1,0,1,0,1,0
51010,3,132.0,126.3,18,0,6,8,5,12.600000,0,...,0,1,1,0,1,0,1,0,1,0


In [45]:
res = dct.predict(d_test)

In [46]:
len(res)

51012

In [47]:
test.id

0         76518
1         76519
2         76520
3         76521
4         76522
          ...  
51007    127525
51008    127526
51009    127527
51010    127528
51011    127529
Name: id, Length: 51012, dtype: int64

In [48]:
result_df = pd.DataFrame(test.id)

In [49]:
result_df["Target"] = res

In [50]:
result_df.head()

,id,Target
0,76518,Dropout
1,76519,Graduate
2,76520,Graduate
3,76521,Enrolled
4,76522,Enrolled


In [51]:
c_d,c_g,c_e = 0,0,0
for i in result_df.Target:
    if i == "Dropout":
        c_d+=1
    elif i == "Graduate":
        c_g+=1
    else:
        c_e+=1

In [52]:
print(f"Number of Dropout Classification = {c_d}")
print(f"Number of Graduate Classification = {c_g}")
print(f"Number of Enrolled Classification = {c_e}")

Number of Dropout Classification = 16530
Number of Graduate Classification = 24216
Number of Enrolled Classification = 10266


In [53]:
# result_df.to_csv("submission.csv",index=False)

# **INTERMEDIATE LEVEL**
>**The approach here is to test few models and compare them then select one which is best among them and hypertune it and use it on data which contains only those features which have high correlation with Target.**<br>
Took help from : https://www.kaggle.com/code/thegodchurch/ml-selecting-model-final-model-xgboost

# COMPARING DECISIONTREECLASSIFIER, CATBOOSTCLASSIFIER & XGBOOSTCLASSIFIER WITH HYPERPARAMETER TUNING & BEST FEATURE SELECTION

> **MORE DATA PRE-PROCESSING**

In [54]:
train = pd.read_csv("/kaggle/input/playground-series-s4e6/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s4e6/test.csv")

In [55]:
train.set_index('id',inplace=True)

In [56]:
to_train = train.copy()

In [57]:
to_train

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
id,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,9238,1,1,126.0,1,1,19,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,17,1,9238,1,1,125.0,1,19,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,1,17,2,9254,1,1,137.0,1,3,19,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,1,1,3,9500,1,1,131.0,1,19,3,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,1,1,2,9500,1,1,132.0,1,19,37,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,1,17,1,9254,1,1,121.0,1,19,1,...,0,6,8,5,10.600000,0,13.9,-0.3,0.79,Graduate
76514,1,1,6,9254,1,1,125.0,1,1,38,...,0,6,9,6,13.875000,0,9.4,-0.8,-3.12,Graduate
76515,5,17,1,9085,1,1,138.0,1,37,37,...,0,5,8,5,11.400000,1,9.4,-0.8,-3.12,Enrolled


In [58]:
label_encoders = {}
label_encoders['Target'] = LabelEncoder()
to_train['Target'] = label_encoders['Target'].fit_transform(to_train['Target'])


In [59]:
cat_data = [
    "Marital status",
    "Application mode",
    "Course",
    "Daytime/evening attendance",
    "Previous qualification",
    "Nacionality",
    "Mother's qualification",
    "Father's qualification",
    "Mother's occupation",
    "Father's occupation",
    "Displaced",
    "Educational special needs",
    "Debtor",
    "Tuition fees up to date",
    "Gender",
    "Scholarship holder",
    "International"
]

In [60]:
# for i in cat_data:
#     to_train[i] = to_train[i].astype('category')

In [61]:
to_train.dtypes

Marital status                                      int64
Application mode                                    int64
Application order                                   int64
Course                                              int64
Daytime/evening attendance                          int64
Previous qualification                              int64
Previous qualification (grade)                    float64
Nacionality                                         int64
Mother's qualification                              int64
Father's qualification                              int64
Mother's occupation                                 int64
Father's occupation                                 int64
Admission grade                                   float64
Displaced                                           int64
Educational special needs                           int64
Debtor                                              int64
Tuition fees up to date                             int64
Gender        

In [62]:
corr_mat = to_train.corr()
corr_with_target = corr_mat['Target'].abs().sort_values(ascending=False)

In [63]:
corr_with_target

Target                                            1.000000
Curricular units 2nd sem (approved)               0.781452
Curricular units 1st sem (approved)               0.725490
Curricular units 2nd sem (grade)                  0.719036
Curricular units 1st sem (grade)                  0.661355
Tuition fees up to date                           0.415691
Scholarship holder                                0.394124
Gender                                            0.330181
Application mode                                  0.324830
Age at enrollment                                 0.320679
Curricular units 2nd sem (enrolled)               0.289165
Curricular units 1st sem (enrolled)               0.263657
Debtor                                            0.248391
Curricular units 2nd sem (evaluations)            0.214951
Admission grade                                   0.172880
Course                                            0.154208
Curricular units 1st sem (evaluations)            0.1523

In [64]:
imp_features = corr_with_target[corr_with_target > 0.1]

In [65]:
imp_cols = imp_features.index.to_list()

In [66]:
df_imp = to_train[imp_cols]

In [67]:
df_imp

,Target,Curricular units 2nd sem (approved),Curricular units 1st sem (approved),Curricular units 2nd sem (grade),Curricular units 1st sem (grade),Tuition fees up to date,Scholarship holder,Gender,Application mode,Age at enrollment,...,Curricular units 2nd sem (evaluations),Admission grade,Course,Curricular units 1st sem (evaluations),Displaced,Previous qualification (grade),Application order,Daytime/evening attendance,Marital status,GDP
id,,,,,,,,,,,,,,,,,,,,,
0,2,6,6,12.428571,14.500000,1,1,0,1,18,...,7,122.6,9238,6,0,126.0,1,1,1,2.02
1,0,0,4,0.000000,11.600000,1,0,0,17,18,...,9,119.8,9238,8,1,125.0,1,1,1,2.02
2,0,0,0,0.000000,0.000000,1,0,1,17,18,...,0,144.7,9254,0,0,137.0,2,1,1,-0.92
3,1,7,7,12.820000,12.591250,1,1,0,1,18,...,11,126.1,9500,9,1,131.0,3,1,1,2.02
4,2,6,6,12.933333,12.933333,1,0,0,1,18,...,12,120.1,9500,12,1,132.0,2,1,1,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,2,5,6,10.600000,10.666667,1,1,0,17,18,...,8,116.5,9254,9,1,121.0,1,1,1,0.79
76514,2,6,4,13.875000,13.000000,1,0,0,1,19,...,9,131.6,9254,22,1,125.0,6,1,1,-3.12
76515,1,5,4,11.400000,12.500000,1,0,0,17,19,...,8,123.3,9085,13,1,138.0,1,1,5,-3.12


In [68]:
scaler = StandardScaler()

# Prepare the data for modeling
X = df_imp.drop('Target',axis=1)
y = df_imp['Target']

In [69]:
imp_cols.remove("Target")

In [70]:
for column in imp_cols:
    if X[column].dtype == 'category':
        label_encoders[column] = LabelEncoder()
        X.loc[:, column] = label_encoders[column].fit_transform(X[column])
    elif X[column].dtype == 'float64':
        X.loc[:, column] = scaler.fit_transform(X[[column]]).astype(float)  # Fit and transform on train



In [71]:
X

,Curricular units 2nd sem (approved),Curricular units 1st sem (approved),Curricular units 2nd sem (grade),Curricular units 1st sem (grade),Tuition fees up to date,Scholarship holder,Gender,Application mode,Age at enrollment,Curricular units 2nd sem (enrolled),...,Curricular units 2nd sem (evaluations),Admission grade,Course,Curricular units 1st sem (evaluations),Displaced,Previous qualification (grade),Application order,Daytime/evening attendance,Marital status,GDP
id,,,,,,,,,,,,,,,,,,,,,
0,6,6,0.505317,0.855619,1,1,0,1,18,6,...,7,-0.220022,9238,6,0,-0.580138,1,1,1,0.933176
1,0,4,-1.735681,0.304727,1,0,0,17,18,6,...,9,-0.442912,9238,8,1,-0.671086,1,1,1,0.933176
2,0,0,-1.735681,-1.898842,1,0,1,17,18,6,...,0,1.539217,9254,0,0,0.420293,2,1,1,-0.372698
3,7,7,0.575895,0.493027,1,1,0,1,18,8,...,11,0.058591,9500,9,1,-0.125396,3,1,1,0.933176
4,6,6,0.596330,0.558010,1,0,0,1,18,7,...,12,-0.419031,9500,12,1,-0.034448,2,1,1,0.178079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,5,6,0.175607,0.127428,1,1,0,17,18,6,...,8,-0.705604,9254,9,1,-1.034880,1,1,1,0.386841
76514,6,4,0.766123,0.570674,1,0,0,1,19,6,...,9,0.496410,9254,22,1,-0.671086,6,1,1,-1.349882
76515,5,4,0.319855,0.475693,1,0,0,17,19,5,...,8,-0.164300,9085,13,1,0.511242,1,1,5,-1.349882


In [72]:
y

id
0        2
1        0
2        0
3        1
4        2
        ..
76513    2
76514    2
76515    1
76516    0
76517    2
Name: Target, Length: 76518, dtype: int64

In [73]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [74]:
models = {
    "Decision Tree": DecisionTreeClassifier(),
    "XGBoost": XGBClassifier(tree_method='hist', device='cuda', use_label_encoder=False, eval_metric='mlogloss'),
    "CatBoost" : CatBoostClassifier()
}

In [75]:
X.dtypes


Curricular units 2nd sem (approved)         int64
Curricular units 1st sem (approved)         int64
Curricular units 2nd sem (grade)          float64
Curricular units 1st sem (grade)          float64
Tuition fees up to date                     int64
Scholarship holder                          int64
Gender                                      int64
Application mode                            int64
Age at enrollment                           int64
Curricular units 2nd sem (enrolled)         int64
Curricular units 1st sem (enrolled)         int64
Debtor                                      int64
Curricular units 2nd sem (evaluations)      int64
Admission grade                           float64
Course                                      int64
Curricular units 1st sem (evaluations)      int64
Displaced                                   int64
Previous qualification (grade)            float64
Application order                           int64
Daytime/evening attendance                  int64


In [76]:
X_train.head()

,Curricular units 2nd sem (approved),Curricular units 1st sem (approved),Curricular units 2nd sem (grade),Curricular units 1st sem (grade),Tuition fees up to date,Scholarship holder,Gender,Application mode,Age at enrollment,Curricular units 2nd sem (enrolled),...,Curricular units 2nd sem (evaluations),Admission grade,Course,Curricular units 1st sem (evaluations),Displaced,Previous qualification (grade),Application order,Daytime/evening attendance,Marital status,GDP
id,,,,,,,,,,,,,,,,,,,,,
12065,5,6,0.568282,0.507353,1,1,0,44,21,6,...,21,1.961117,9085,8,1,1.602622,1,1,1,-0.372698
17210,3,3,0.247731,0.317391,1,0,1,17,20,6,...,12,-0.251864,9254,10,1,-0.489190,2,1,1,-1.767406
60954,5,4,0.464103,0.285730,1,1,0,1,19,6,...,6,0.536212,9773,9,1,0.693138,1,1,1,0.178079
2322,0,0,-1.735681,-1.898842,1,0,1,1,18,0,...,0,-0.610080,171,0,0,-2.035311,1,1,1,0.831016
374,1,0,0.247731,-1.898842,1,0,1,39,27,6,...,13,-2.019063,9238,12,0,0.065595,1,1,1,-1.349882


In [77]:
train

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
id,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,9238,1,1,126.0,1,1,19,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,17,1,9238,1,1,125.0,1,19,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,1,17,2,9254,1,1,137.0,1,3,19,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,1,1,3,9500,1,1,131.0,1,19,3,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,1,1,2,9500,1,1,132.0,1,19,37,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,1,17,1,9254,1,1,121.0,1,19,1,...,0,6,8,5,10.600000,0,13.9,-0.3,0.79,Graduate
76514,1,1,6,9254,1,1,125.0,1,1,38,...,0,6,9,6,13.875000,0,9.4,-0.8,-3.12,Graduate
76515,5,17,1,9085,1,1,138.0,1,37,37,...,0,5,8,5,11.400000,1,9.4,-0.8,-3.12,Enrolled


In [78]:
X.head()

,Curricular units 2nd sem (approved),Curricular units 1st sem (approved),Curricular units 2nd sem (grade),Curricular units 1st sem (grade),Tuition fees up to date,Scholarship holder,Gender,Application mode,Age at enrollment,Curricular units 2nd sem (enrolled),...,Curricular units 2nd sem (evaluations),Admission grade,Course,Curricular units 1st sem (evaluations),Displaced,Previous qualification (grade),Application order,Daytime/evening attendance,Marital status,GDP
id,,,,,,,,,,,,,,,,,,,,,
0,6,6,0.505317,0.855619,1,1,0,1,18,6,...,7,-0.220022,9238,6,0,-0.580138,1,1,1,0.933176
1,0,4,-1.735681,0.304727,1,0,0,17,18,6,...,9,-0.442912,9238,8,1,-0.671086,1,1,1,0.933176
2,0,0,-1.735681,-1.898842,1,0,1,17,18,6,...,0,1.539217,9254,0,0,0.420293,2,1,1,-0.372698
3,7,7,0.575895,0.493027,1,1,0,1,18,8,...,11,0.058591,9500,9,1,-0.125396,3,1,1,0.933176
4,6,6,0.596330,0.558010,1,0,0,1,18,7,...,12,-0.419031,9500,12,1,-0.034448,2,1,1,0.178079


In [79]:
info_model = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    cls_report = classification_report(y_val, y_pred)
    info_model[name] = [accuracy,cls_report]

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:35:00] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Learning rate set to 0.097601
0:	learn: 0.9939812	total: 78ms	remaining: 1m 17s
1:	learn: 0.9158207	total: 98.1ms	remaining: 49s
2:	learn: 0.8517306	total: 121ms	remaining: 40.3s
3:	learn: 0.8001679	total: 142ms	remaining: 35.4s
4:	learn: 0.7572505	total: 163ms	remaining: 32.4s
5:	learn: 0.7210559	total: 184ms	remaining: 30.4s
6:	learn: 0.6904899	total: 207ms	remaining: 29.3s
7:	learn: 0.6652231	total: 228ms	remaining: 28.3s
8:	learn: 0.6424575	total: 250ms	remaining: 27.5s
9:	learn: 0.6222285	total: 271ms	remaining: 26.8s
10:	learn: 0.6055214	total: 295ms	remaining: 26.5s
11:	learn: 0.5900525	total: 315ms	remaining: 26s
12:	learn: 0.5768523	total: 337ms	remaining: 25.6s
13:	learn: 0.5651444	total: 357ms	remaining: 25.2s
14:	learn: 0.5549576	total: 377ms	remaining: 24.7s
15:	learn: 0.5461230	total: 398ms	remaining: 24.5s
16:	learn: 0.5380803	total: 420ms	remaining: 24.3s
17:	learn: 0.5308423	total: 441ms	remaining: 24.1s
18:	learn: 0.5247877	total: 462ms	remaining: 23.9s
19:	learn: 0.5

In [80]:
for key,value in info_model.items():
    print(f"MODEL NAME : {key}",end=" ")
    print(f"ACCURACY SCORE = {value[0]}")
    print(f"{value[1]}")

MODEL NAME : Decision Tree ACCURACY SCORE = 0.7384997386304234
              precision    recall  f1-score   support

           0       0.77      0.80      0.79      5028
           1       0.47      0.47      0.47      3017
           2       0.82      0.81      0.82      7259

    accuracy                           0.74     15304
   macro avg       0.69      0.69      0.69     15304
weighted avg       0.74      0.74      0.74     15304

MODEL NAME : XGBoost ACCURACY SCORE = 0.8298484056455828
              precision    recall  f1-score   support

           0       0.90      0.83      0.86      5028
           1       0.65      0.60      0.63      3017
           2       0.86      0.92      0.89      7259

    accuracy                           0.83     15304
   macro avg       0.80      0.79      0.79     15304
weighted avg       0.83      0.83      0.83     15304

MODEL NAME : CatBoost ACCURACY SCORE = 0.8314166231050706
              precision    recall  f1-score   support

     

In [81]:
for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5)
    print(f"{name} Cross-validation Accuracy: {scores.mean():.4f} (+/- {scores.std():.4f})")

Decision Tree Cross-validation Accuracy: 0.7410 (+/- 0.0029)
XGBoost Cross-validation Accuracy: 0.8267 (+/- 0.0016)
Learning rate set to 0.097601
0:	learn: 0.9941967	total: 23.8ms	remaining: 23.8s
1:	learn: 0.9141543	total: 44ms	remaining: 21.9s
2:	learn: 0.8522551	total: 67.7ms	remaining: 22.5s
3:	learn: 0.8011057	total: 88.5ms	remaining: 22s
4:	learn: 0.7579558	total: 110ms	remaining: 22s
5:	learn: 0.7220742	total: 131ms	remaining: 21.8s
6:	learn: 0.6912927	total: 154ms	remaining: 21.8s
7:	learn: 0.6660583	total: 176ms	remaining: 21.8s
8:	learn: 0.6436527	total: 198ms	remaining: 21.8s
9:	learn: 0.6233238	total: 219ms	remaining: 21.7s
10:	learn: 0.6062906	total: 241ms	remaining: 21.6s
11:	learn: 0.5911345	total: 262ms	remaining: 21.6s
12:	learn: 0.5780502	total: 282ms	remaining: 21.4s
13:	learn: 0.5660497	total: 302ms	remaining: 21.2s
14:	learn: 0.5556701	total: 322ms	remaining: 21.2s
15:	learn: 0.5464404	total: 344ms	remaining: 21.2s
16:	learn: 0.5381641	total: 367ms	remaining: 21.2s

> **CatCoostClassifier is working better than the other two**

In [82]:
# Define the objective function for Optuna
def h_tuning(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 32, 100),
        'thread_count': trial.suggest_int('thread_count', 1, 4)
    }
    
    model = CatBoostClassifier(**params, verbose=0)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=30, verbose=0)
    
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    
    return accuracy

In [83]:
# Create an Optuna study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(h_tuning, n_trials=50)

[I 2024-06-27 10:37:05,663] A new study created in memory with name: no-name-6678154f-5d37-4586-b117-6eef12ab6cdc
[I 2024-06-27 10:37:52,500] Trial 0 finished with value: 0.8301751176163095 and parameters: {'iterations': 992, 'depth': 6, 'learning_rate': 0.05331965594667198, 'l2_leaf_reg': 8, 'border_count': 87, 'thread_count': 1}. Best is trial 0 with value: 0.8301751176163095.
[I 2024-06-27 10:38:08,183] Trial 1 finished with value: 0.8320047046523784 and parameters: {'iterations': 811, 'depth': 7, 'learning_rate': 0.06753424506817868, 'l2_leaf_reg': 2, 'border_count': 38, 'thread_count': 3}. Best is trial 1 with value: 0.8320047046523784.
[I 2024-06-27 10:38:53,319] Trial 2 finished with value: 0.8279534762153685 and parameters: {'iterations': 841, 'depth': 8, 'learning_rate': 0.023559214362709604, 'l2_leaf_reg': 4, 'border_count': 51, 'thread_count': 3}. Best is trial 1 with value: 0.8320047046523784.
[I 2024-06-27 10:39:16,292] Trial 3 finished with value: 0.829260324098275 and pa

In [84]:
# Best trial
best_trial = study.best_trial
print("Best Parameters:", best_trial.params)
print("Best Score:", best_trial.value)

Best Parameters: {'iterations': 887, 'depth': 5, 'learning_rate': 0.07071738636523639, 'l2_leaf_reg': 3, 'border_count': 96, 'thread_count': 3}
Best Score: 0.8327888133821223


In [85]:

# Use the best parameters to fit the final model
best_params = best_trial.params
best_model = CatBoostClassifier(**best_params)
best_model.fit(X_train, y_train)

0:	learn: 1.0229754	total: 21.8ms	remaining: 19.3s
1:	learn: 0.9609633	total: 43.5ms	remaining: 19.3s
2:	learn: 0.9085044	total: 64.3ms	remaining: 18.9s
3:	learn: 0.8641597	total: 84.7ms	remaining: 18.7s
4:	learn: 0.8256856	total: 106ms	remaining: 18.6s
5:	learn: 0.7907868	total: 126ms	remaining: 18.5s
6:	learn: 0.7613150	total: 147ms	remaining: 18.5s
7:	learn: 0.7349298	total: 169ms	remaining: 18.5s
8:	learn: 0.7126195	total: 191ms	remaining: 18.6s
9:	learn: 0.6918781	total: 214ms	remaining: 18.8s
10:	learn: 0.6729467	total: 237ms	remaining: 18.9s
11:	learn: 0.6562385	total: 258ms	remaining: 18.8s
12:	learn: 0.6417734	total: 280ms	remaining: 18.8s
13:	learn: 0.6281464	total: 300ms	remaining: 18.7s
14:	learn: 0.6154021	total: 321ms	remaining: 18.6s
15:	learn: 0.6042134	total: 343ms	remaining: 18.7s
16:	learn: 0.5939196	total: 364ms	remaining: 18.6s
17:	learn: 0.5845901	total: 384ms	remaining: 18.5s
18:	learn: 0.5758355	total: 404ms	remaining: 18.5s
19:	learn: 0.5682906	total: 425ms	rem

In [86]:
y_pred = best_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
cls_report = classification_report(y_val, y_pred)
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{cls_report}")

Accuracy: 0.832723470987977
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.83      0.87      5028
           1       0.65      0.61      0.63      3017
           2       0.85      0.92      0.89      7259

    accuracy                           0.83     15304
   macro avg       0.80      0.79      0.80     15304
weighted avg       0.83      0.83      0.83     15304



In [87]:
to_test = test[imp_cols]

In [88]:
res_preds = best_model.predict(to_test)

In [89]:
res_preds

array([[0],
       [2],
       [2],
       ...,
       [0],
       [0],
       [0]])

In [90]:
result = pd.DataFrame({'id': pd.read_csv("/kaggle/input/playground-series-s4e6/test.csv").id, 'Target': label_encoders['Target'].inverse_transform(res_preds.astype(int))})

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [91]:
result.to_csv("submission.csv",index=False)